In [3]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras.models import load_model
from keras import backend as K
import numpy as np
import nbimporter
from Train_Dataset_Generation import *

Importing Jupyter notebook from Train_Dataset_Generation.ipynb
Importing Jupyter notebook from Dataset_Preparation.ipynb


### Hyperparameters

In [5]:
n_classes = 133
n_rows = 32
n_cols = 32
batch_size = 16
epochs = 10
learning_rate = 0.1
decay = 1e-6
momentum = .9

### Creating The Model

In [34]:
def create_model():
    # conv-conv-pool ==> conv-conv-pool ==> dense ==>dense
    print('Building Model..')
    model = Sequential()
    
    model.add(Conv2D(32,(3,3),input_shape=(n_rows,n_cols,1),activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(64,(3,3),activation='relu'))
    
    model.add(MaxPooling2D((2,2),strides=(2,2)))
    model.add(ZeroPadding2D((1,1)))

    model.add(Conv2D(64,(3,3),activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Conv2D(64,(3,3),activation='relu')) 
    
    model.add(MaxPooling2D((2,2),strides=(2,2)))
    
    model.add(Flatten())
    
    model.add(Dense(2018, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(n_classes, activation='softmax'))
    
    sgd = SGD(lr = learning_rate, decay = decay, momentum = momentum,nesterov=True)
    model.compile(optimizer = sgd, loss='categorical_crossentropy')
    
    print('Model Generated')
    
    return model

In [41]:
def trainCNN(X_train,y_train):
    
    model = create_model()
    print('Training Started . Please Wait ......')
    model.fit(X_train, y_train, epochs = epochs, batch_size = batch_size, verbose = 1)
    print('Training Completed')
    
    return model

In [42]:
def saveWeights(model,name):
    model.save('Models/'+name+'.h5')
    print('Model Saved to Models/'+name+'.h5')

In [44]:
def loadSavedWeight(name):
    model = load_model('Models/'+name+'.h5')
    return model

In [ ]:
def MalayalamCharacterRecognition():
    
    X_train,y_train,X_test,y_test = get